In [ ]:
# Colab-ready KPSAT Fast (paste this into a Colab cell).
# Edit only FILEPATH below to point to your .cnf / .cnf.xz / .cnf.gz / .zip file in Drive or local.

import os, lzma, gzip, zipfile
import numpy as np

# ----------------------------- USER: set file path here -----------------------------
# Example Drive path after mounting: "/content/drive/MyDrive/formulas/my.cnf.xz"
FILEPATH = "/content/04ded94454830d4ea960327e8b91f5a3-mdp-28-14-sat (1).cnf.xz"
# ------------------------------------------------------------------------------

# ----------------- (Optional) Mount Google Drive if file is in Drive -----------------
# If your FILEPATH is in Drive, run this once (uncomment) to mount:
# from google.colab import drive
# drive.mount('/content/drive')
# ------------------------------------------------------------------------------

# ---------------- CNF reader (supports compressed .xz, .gz, .zip) ----------------
def read_cnf_file(filename):
    if filename.endswith(".xz"):
        with lzma.open(filename, "rt") as f:
            lines = f.readlines()
    elif filename.endswith(".gz"):
        with gzip.open(filename, "rt") as f:
            lines = f.readlines()
    elif filename.endswith(".zip"):
        with zipfile.ZipFile(filename, 'r') as z:
            name = z.namelist()[0]
            with z.open(name) as f:
                lines = [l.decode('utf-8') for l in f.readlines()]
    else:
        with open(filename, "r") as f:
            lines = f.readlines()

    clauses = []
    n_vars = 0
    for raw in lines:
        s = raw.strip()
        if not s or s.startswith("c"):
            continue
        if s.startswith("p cnf"):
            parts = s.split()
            if len(parts) >= 3:
                n_vars = int(parts[2])
        else:
            try:
                lits = list(map(int, s.split()))
                if not lits:
                    continue
                if lits[-1] == 0:
                    lits = lits[:-1]
                if lits:
                    clauses.append(lits)
            except ValueError:
                continue
    return n_vars, clauses

# ---------------- Build padded clause matrices ----------------
def build_clause_matrices(clauses):
    if not clauses:
        return np.zeros((0,0), dtype=np.int32), np.zeros((0,0), dtype=np.int8), np.zeros((0,0), dtype=np.float32)
    m = len(clauses)
    k = max(len(c) for c in clauses)
    indices = -np.ones((m, k), dtype=np.int32)
    signs = np.zeros((m, k), dtype=np.int8)
    mask = np.zeros((m, k), dtype=np.float32)
    for i, clause in enumerate(clauses):
        for j, lit in enumerate(clause):
            indices[i, j] = abs(lit) - 1
            signs[i, j] = 1 if lit > 0 else -1
            mask[i, j] = 1.0
    return indices, signs, mask

# ---------------- Vectorized solver (Adam) ----------------
def kpsat_fast(values, indices, signs, mask,
               eps=1e-6, max_iter=2000, lr=0.02,
               beta1=0.9, beta2=0.999, eps_adam=1e-8, dtype=np.float32):
    values = np.asarray(values, dtype=dtype)
    n = values.shape[0]
    if indices.size == 0:
        a = np.zeros(n, dtype=dtype)
        assignment = {i+1: bool(int(round(float(ai)))) for i, ai in enumerate(a)}
        return assignment, float(np.dot(values, a)), [0.0]

    m, k = indices.shape
    idx_pad = indices.copy()
    idx_pad[idx_pad < 0] = 0
    signs_f = signs.astype(dtype)
    mask_f = mask.astype(dtype)

    a = np.random.rand(n).astype(dtype)
    m_t = np.zeros_like(a)
    v_t = np.zeros_like(a)
    phi_hist = []
    range_k = range(k)

    for t in range(1, max_iter + 1):
        a_vals = a[idx_pad]                           # (m,k)
        lit_vals = np.where(signs_f > 0, a_vals, 1.0 - a_vals) * mask_f
        one_minus = 1.0 - lit_vals
        clause_prod = np.prod(one_minus, axis=1)
        clause_val = 1.0 - clause_prod
        resid = 1.0 - clause_val
        phi = float(np.sum(resid * resid))
        phi_hist.append(phi)
        if phi < eps:
            break

        prefix = np.cumprod(one_minus, axis=1)
        suffix = np.cumprod(one_minus[:, ::-1], axis=1)[:, ::-1]

        partials = np.empty_like(one_minus)
        for j in range_k:
            if j == 0:
                pref = 1.0
            else:
                pref = prefix[:, j-1]
            if j == k - 1:
                suf = 1.0
            else:
                suf = suffix[:, j+1]
            partials[:, j] = pref * suf

        coef = (-2.0 * resid)[:, None] * partials * signs_f * mask_f   # (m,k)

        grad = np.zeros(n, dtype=dtype)
        for j in range_k:
            idx_col = idx_pad[:, j]
            contrib = coef[:, j]
            valid = mask_f[:, j].astype(bool)
            if np.any(valid):
                np.add.at(grad, idx_col[valid], contrib[valid])

        # Adam update
        m_t = beta1 * m_t + (1 - beta1) * grad
        v_t = beta2 * v_t + (1 - beta2) * (grad * grad)
        m_hat = m_t / (1 - beta1 ** t)
        v_hat = v_t / (1 - beta2 ** t)
        step = lr * m_hat / (np.sqrt(v_hat) + eps_adam)
        a -= step
        np.clip(a, 0.0, 1.0, out=a)

    assignment = {i + 1: bool(int(round(float(ai)))) for i, ai in enumerate(a)}
    subset_sum = float(np.dot(values, a))
    return assignment, subset_sum, phi_hist

# ---------------- Save output (silent) ----------------
def save_cnf_solution(filename, assignment):
    base = os.path.splitext(os.path.basename(filename))[0]
    out_file = f"{base}.resolved.cnf"
    with open(out_file, "w") as f:
        f.write("c Resolved CNF generated by KPSAT_FAST\n")
        n_vars = len(assignment)
        f.write(f"p cnf {n_vars} {n_vars}\n")
        for i in range(1, n_vars + 1):
            lit = i if assignment[i] else -i
            f.write(f"{lit} 0\n")
    return out_file

def save_unsat_marker(filename):
    base = os.path.splitext(os.path.basename(filename))[0]
    out_file = f"{base}.unsat"
    with open(out_file, "w") as f:
        f.write("UNSAT\n")
    return out_file

# ---------------- Main minimal pipeline ----------------
def kpsat_colab_minimal(filepath, eps=1e-6, max_iter=2000, lr=0.02, phi_threshold=1e-4):
    n_vars, clauses = read_cnf_file(filepath)
    indices, signs, mask = build_clause_matrices(clauses)
    values = np.arange(1, max(1, n_vars) + 1, dtype=np.float32)
    assignment, subset_sum, phi_hist = kpsat_fast(values, indices, signs, mask,
                                                  eps=eps, max_iter=max_iter, lr=lr)
    is_sat_prob = (phi_hist[-1] < phi_threshold) if phi_hist else True
    if is_sat_prob:
        save_cnf_solution(filepath, assignment)
        print("SAT")
    else:
        save_unsat_marker(filepath)
        print("UNSAT")

# ---------------- Run with provided FILEPATH ----------------
if __name__ == "__main__":
    if not os.path.exists(FILEPATH):
        raise FileNotFoundError(f"File not found: {FILEPATH} -- mount Drive or upload file to Colab and set FILEPATH.")
    # You can tweak eps, max_iter, lr, phi_threshold as needed:
    kpsat_colab_minimal(FILEPATH, eps=1e-6, max_iter=1500, lr=0.02, phi_threshold=1e-4)
